# Using a regression neural network to predict housing prices in Hyderabad

### Importing packages

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras

# Setting random seeds to get reproducible results
np.random.seed(0)
import tensorflow as tf
tf.random.set_seed(1)

### Loading the dataset
For the next two cells, please run only one of them.
- Run the first cell if you cloned the Github Repo
- Run the second cell if you opened this as a Google Colab

In [ ]:
# IMPORTANT: ONLY RUN THIS CELL IF YOU HAVE CLONED THE REPO
housing = pd.read_csv('Hyderabad.csv')
housing

,Price,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,...,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
0,6968000,1340,Nizampet,2,0,0,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
1,29000000,3498,Hitech City,4,0,0,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
2,6590000,1318,Manikonda,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5739000,1295,Alwal,3,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,5679000,1145,Kukatpally,2,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,11000000,1460,Nacharam,2,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2514,26000000,1314,Manikonda,2,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2515,13300000,2625,Madhapur,3,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2516,10800000,2050,Hitech City,3,0,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9


In [2]:
# IMPORTANT: ONLY RUN THIS CELL IF YOU ARE WORKING ON A COLAB

url = "https://raw.githubusercontent.com/luisguiserrano/manning/master/Chapter_10_Neural_Networks/Hyderabad.csv"
housing = pd.read_csv(url)
housing

### Preprocessing the data

In [3]:
features = housing.drop(['Location', 'Price'], axis=1)
features

,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,...,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
0,1340,2,0,0,1,1,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
1,3498,4,0,0,1,1,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
2,1318,2,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1295,3,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1145,2,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,1460,2,1,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2514,1314,2,1,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2515,2625,3,1,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2516,2050,3,0,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9


In [4]:
labels = housing['Price']
labels

,Price
0,6968000
1,29000000
2,6590000
3,5739000
4,5679000
...,...
2513,11000000
2514,26000000
2515,13300000
2516,10800000


### Building and training the neural network

In [5]:
# Imports
#import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
#from tensorflow.keras.layers import Dense, Dropout, Activation
#from tensorflow.keras.optimizers import SGD

# Building the model
model = Sequential()
model.add(Dense(38, activation='relu', input_shape=(38,)))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1))

# Compiling the model. The metrics flag is added for the model to report the root mean squared error at each epoch.
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[keras.metrics.RootMeanSquaredError()])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 38)             │         1,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 38)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,795 (57.79 KB)

 Trainable params: 14,795 (57.79 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.fit(features, labels, epochs=10, batch_size=10)

Epoch 1/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 181014534029312.0000 - root_mean_squared_error: 13430808.0000
Epoch 2/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 47454414176256.0000 - root_mean_squared_error: 6851713.5000
Epoch 3/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 38791158956032.0000 - root_mean_squared_error: 6197422.5000
Epoch 4/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 37403024687104.0000 - root_mean_squared_error: 6086184.5000
Epoch 5/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 37183750668288.0000 - root_mean_squared_error: 6070181.5000
Epoch 6/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 37893452070912.0000 - root_mean_squared_error: 6121180.5000
Epoch 7/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37565721739264.0000 - root_mean_squared_error: 6096579.0000
Epoch 8/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35391130304512.0000 - root_mean_squared_error: 5919994.5000
Epoch 9/10
252/252 ━━━━━━━━━━━━━━━━━━━

### Evaluating the model and making predictions

In [7]:
model.evaluate(features, labels)

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 17141183021056.0000 - root_mean_squared_error: 4098900.5000  


[30672076931072.0, 5538237.5]

In [8]:
model.predict(features)

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


array([[ 8700264.],
       [22709732.],
       [ 8553653.],
       ...,
       [16902904.],
       [13163288.],
       [11572599.]], dtype=float32)

In [9]:
labels

,Price
0,6968000
1,29000000
2,6590000
3,5739000
4,5679000
...,...
2513,11000000
2514,26000000
2515,13300000
2516,10800000
